# Compute definition

In [1]:
import d2ltvm
import numpy as np
import tvm
from tvm import te

/staff/qiaoliang/ACSA科研项目/tvm_learning/d2ltvm/d2ltvm.py:153: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if conv_type is 'depthwise':
/staff/qiaoliang/ACSA科研项目/tvm_learning/d2ltvm/d2ltvm.py:184: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  PaddedX = padding(X, ph, pw) if ph | pw is not 0 else X


successd...



In [3]:
from d2ltvm import padding, conv_out_size
def depthwise_conv(ic, nh, nw, kh, kw, ph=0, pw=0, sh=1, sw=1):
    """ Convolution
        ic : number of channels for both input and output
        nh, nw : input width and height
        kh, kw : kernel width and height
        ph, pw : height and width padding sizes, default 0
        sh, sw : height and width strides, default 1
    """
    # reduction axis
    rkh = te.reduce_axis((0, kh), name='rkh')
    rkw = te.reduce_axis((0, kw), name='rkw')
    # output height and weights
    oh = conv_out_size(nh, kh, ph, sh)
    ow = conv_out_size(nw, kw, pw, sw)
    # pad X and then compute Y
    X = te.placeholder((ic, nh, nw), name='X')
    K = te.placeholder((ic, 1, kh, kw), name='Y')
    PaddedX = padding(X, ph, pw) if (ph | pw) != 0 else X
    Y = te.compute((ic, oh, ow),
                  lambda c, i, j: te.sum(PaddedX[c, i * sh + rkh, j * sw + rkw] * K[c, 0, rkh, rkw],
                    axis=[rkh, rkw]), name='Y')
    return X, K, Y, PaddedX

In [4]:
ic, n, k, p, s = 256, 12, 3, 1, 1
X, K, Y, _ = depthwise_conv(ic, n, n, k, k, p, p, s, s)
sch = te.create_schedule(Y.op)
print(tvm.lower(sch, [X, K, Y], simple_mode=True))

@main = primfn(X_1: handle, Y_2: handle, Y_3: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {X: Buffer(X_2: Pointer(float32), float32, [36864], []),
             Y: Buffer(Y_4: Pointer(float32), float32, [2304], []),
             Y_1: Buffer(Y_5: Pointer(float32), float32, [36864], [])}
  buffer_map = {X_1: X, Y_2: Y, Y_3: Y_1}
  preflattened_buffer_map = {X_1: X_3: Buffer(X_2, float32, [256, 12, 12], []), Y_2: Y_6: Buffer(Y_4, float32, [256, 1, 3, 3], []), Y_3: Y_7: Buffer(Y_5, float32, [256, 12, 12], [])} {
  allocate(PaddedX: Pointer(global float32), float32, [50176]), storage_scope = global {
    for (i0: int32, 0, 256) {
      for (i1: int32, 0, 14) {
        for (i2: int32, 0, 14) {
          PaddedX_1: Buffer(PaddedX, float32, [50176], [])[(((i0*196) + (i1*14)) + i2)] = @tir.if_then_else(((((i1 < 1) || (13 <= i1)) || (i2 < 1)) || (13 <= i2)), 0f32, X[((((i0*144) + (i1*12)) + i2) - 13)], dtype=float32)
        }

In [10]:
mod = tvm.build(sch, [X, K, Y])
data, weight, out = d2ltvm.get_conv_data(ic, ic, n, k, p, s, tvm.nd.array, 'depthwise')
mod(data, weight, out)

In [14]:
import torch
def get_conv_data_torch(oc, ic, n, k, p, s, ctx='cpu', conv_type='direct'):
    device = torch.device(ctx)
    data, weight, _ = d2ltvm.get_conv_data(oc, ic, n, k, p, s, constructor=lambda x: torch.tensor(x, device=device), conv_type='depthwise')
    data = data[None, ...]
    bias = torch.zeros(oc, device=device)
    return data, weight, bias

In [15]:
def depthwise_conv_torch(data, weight, bias, p, s):
    return torch.nn.functional.conv2d(data, weight, bias, s, p, groups=data.shape[1])

In [18]:
data1, weight1, bias1 = get_conv_data_torch(ic, ic, n, k, p, s, conv_type='depthwise')
out1 = depthwise_conv_torch(data1, weight1, bias1, p, s)
np.testing.assert_allclose(out.asnumpy(), out1[0].numpy(), atol=1e-5)

# Summary
1.Depthwise convolution, together with pointwise convolution, can save a lot of computation and memory compared to normal 2-D convolution \
2.Depthwise convolution takes kernels in 3-D, while normal 2-D convolution takes kernels in 4-D